### Reading MNIST

In [1]:
import pandas as pd
import numpy as np


train = pd.read_csv("./data/mnist_train.csv")
test = pd.read_csv("./data/mnist_test.csv")

# Reestructuring
train_label, train_image = train["label"].to_numpy(), train[[x for x in train.columns if x != "label"]].to_numpy().reshape((-1, 1, 28, 28))/255
test_label, test_image = train["label"].to_numpy(), train[[x for x in train.columns if x != "label"]].to_numpy().reshape((-1, 1, 28, 28))/255

# One-hotting
one_hot = np.zeros((len(train_label), 10))
for label in range(train_label.shape[0]):
    one_hot[label, train_label[label]] = 1
train_label = one_hot

one_hot = np.zeros((len(test_label), 10))
for label in range(test_label.shape[0]):
    one_hot[label, test_label[label]] = 1
test_label = one_hot

### Creating The Model

In [2]:
import framework as fm

class Model(fm.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = fm.Conv2d(1, 6, 5, 1, bias = False)
        self.relu1 = fm.ReLu()
        self.maxpool1 = fm.MaxPool2d(2, 6, 2)
        self.conv2 = fm.Conv2d(6, 16, 5, 1, bias = False)
        self.maxpool2 = fm.MaxPool2d(2, 16, 2)
        self.lin1 = fm.Linear(256, 84, bias=False)
        self.lin2 = fm.Linear(84, 10, bias=False)


    def forward(self, x):
        x = self.conv1.forward(x)
        x = self.relu1.forward(x)
        x = self.maxpool1.forward(x)
        x = self.conv2.forward(x) 
        x = self.maxpool2.forward(x)
        x = x.reshape((-1, 256))
        x = self.lin1.forward(x) 
        x = self.lin2.forward(x)
        return x

model = Model()
loss_fn = fm.CrossEntropy()
optim = fm.Adadelta(model.get_parameters())


#### Training The Model

In [3]:
batch_size = 128
num_batches = int(len(train_label)/batch_size)
epochs = 1


for epoch in range(epochs):

    print(f"Epoch {epoch+1}\n-------------------------------")

    total_loss = 0
    correct = 0

    for batch in range(num_batches):
        batch_start, batch_end = (batch * batch_size, (batch + 1) * batch_size)

        inpt = fm.Tensor(train_image[batch_start:batch_end])
        label = fm.Tensor(train_label[batch_start:batch_end])
        
        pred = model.forward(inpt)

        pred_nums = np.argmax(pred.data, axis=1)
        label_nums = np.argmax(label.data, axis=1)

        correct += (pred_nums == label_nums).sum()
        
        loss = loss_fn.forward(pred, label)

        total_loss += loss 

        if batch % int(num_batches/10) == 0:
            print(f"loss: {loss} [{(batch+1)*batch_size}/{batch_size*num_batches}]")   
            correct = 0
      
        loss_fn.backward()

        optim.step()

    print("Avg error: ", total_loss / num_batches, "\n")  

Epoch 1
-------------------------------
loss: 11.498363494873047 [128/59904]
loss: 1.113430142402649 [6016/59904]
loss: 0.769553005695343 [11904/59904]
loss: 0.6939627528190613 [17792/59904]
loss: 0.48256611824035645 [23680/59904]
loss: 0.47543007135391235 [29568/59904]
loss: 0.29138076305389404 [35456/59904]
loss: 0.7248424887657166 [41344/59904]
loss: 0.2649429738521576 [47232/59904]
loss: 0.35559549927711487 [53120/59904]
loss: 0.11969241499900818 [59008/59904]
Avg error:  0.83835423 



### Saving model

In [10]:
model.save("prueba2")

Object saved successfully.


### Load Model

In [3]:
import pickle
import framework as fm 
import numpy as np 
import pandas as np

with open('prueba2.pkl', 'rb') as file:
    model = pickle.load(file)

print("Object loaded successfully.")

Object loaded successfully.


#### Test Accuracy

In [5]:
# Testing
correct = 0
batch_size = 128
num_batches = int(len(train_label)/batch_size)
for batch in range(num_batches):
        batch_start, batch_end = (batch * batch_size, (batch + 1) * batch_size)

        inpt = fm.Tensor(test_image[batch_start:batch_end])
        label = fm.Tensor(test_label[batch_start:batch_end])
        
        pred = model.forward(inpt)

        pred_nums = np.argmax(pred.data, axis=1)
        label_nums = np.argmax(label.data, axis=1)

        correct += (pred_nums == label_nums).sum()

print("Accuracy:", correct / (batch_size * num_batches) * 100, "%")

NameError: name 'train_label' is not defined

#### Creation of API

In [4]:
import uvicorn
from fastapi import Request, FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/predict")
async def predict(request: Request):
    res = await request.json()
    picture = fm.Tensor(np.array(res).reshape((1, 1, 28, 28)))
    pred = model.forward(picture)
    num = int(np.argmax(pred.data))
    return num

if __name__ == "__main__":
    
    nest_asyncio.apply()  # Allow asyncio.run to work in a running loop
    uvicorn.run(app, host="127.0.0.1", port=3838)

INFO:     Started server process [13172]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3838 (Press CTRL+C to quit)


INFO:     127.0.0.1:53977 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\eric\AppData\Local\Programs\Python\Python312-arm64\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eric\AppData\Local\Programs\Python\Python312-arm64\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eric\AppData\Local\Programs\Python\Python312-arm64\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\eric\AppData\Local\Programs\Python\Python312-arm64\Lib\site-packages\starlette\applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\eric\AppData\Local\Programs\